In [3]:
import pandas as pd
import numpy as np
import datetime as datetime
import os
os.getcwd()
dir = "/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data/"
os.chdir(dir)
os.getcwd()

'/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data'

In [58]:
keys = ['8to9', '9to10', '10to11', '11to12','12to13','13to14','14to15','15to16','16to17','17to18','18to19','19to20']

ec_accptresponse = ['15 to 30 minutes', '5 to 15 minutes', 'Less than 5 minutes']
eventcontingent_dictionary = {'Less than 5 minutes': 2.5, 
                              '15 to 30 minutes': 17.5, 
                              '5 to 15 minutes': 10
                             } 

In [300]:
# read data
ec_original_cloud_ids = [201, 221, 229] 
eod_original_cloud_ids = [201, 203, 206, 221, 229] 

contingent_ema = pd.read_csv(dir + 'eventcontingent-ema.csv')
contingent_ema = contingent_ema.drop(['offset'], axis = 1)
eod_ema = pd.read_csv(dir + 'eod-ema.csv')
eod_ema = eod_ema.drop(['offset'], axis = 1)

contingent_ema_alternative = pd.read_csv(dir + 'eventcontingent-ema-alternative.csv')
eod_ema_alternative = pd.read_csv(dir + 'eod-ema-alternative.csv')

contingent_ema_backup = pd.read_csv(dir + 'eventcontingent-ema-backup.csv')
eod_ema_backup = pd.read_csv(dir + 'eod-ema-backup.csv')

temp_contingent_original = contingent_ema[contingent_ema['participant_id'].isin(ec_original_cloud_ids)]
temp_contingent_alt = contingent_ema_alternative[~contingent_ema_alternative['participant_id'].isin(ec_original_cloud_ids)]

contingent_complete = pd.concat([temp_contingent_original, temp_contingent_alt, contingent_ema_backup])

temp_eod_original = eod_ema[eod_ema['participant_id'].isin(eod_original_cloud_ids)]
temp_eod_alt = eod_ema_alternative[~eod_ema_alternative['participant_id'].isin(eod_original_cloud_ids)]

eod_complete = pd.concat([temp_eod_original, temp_eod_alt, eod_ema_backup])



In [297]:
# Make a list of all contingent event-times between 8AM and 8PM
# Throw away observations for 'when_smoke' is nan or 
# 'More than 30 minutes' to ensure we can calculate a meaningful 
# quantity.
days_smoked = {}
for index, row in contingent_complete.iterrows():
    count += 1
    try:
        time = datetime.datetime.strptime(row['date'], '%m/%d/%y %H:%M')
    except:
        time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
    if row['when_smoke'] in ec_accptresponse:
        time = time - datetime.timedelta(minutes=eventcontingent_dictionary[row['when_smoke']])
    date = (time.year, time.month, time.day, time.hour)
    if row['participant_id'] not in days_smoked:
        days_smoked[row['participant_id']] = set()
    if 8 <= date[3] < 20 and row['when_smoke'] in ec_accptresponse:        
        days_smoked[row['participant_id']].add(date)

days_smoked

{201: {(2017, 6, 23, 18), (2017, 6, 24, 15), (2017, 6, 24, 19)},
 202: {(2017, 6, 26, 13),
  (2017, 6, 27, 9),
  (2017, 6, 27, 12),
  (2017, 7, 6, 15)},
 204: {(2017, 8, 14, 16),
  (2017, 8, 14, 19),
  (2017, 8, 15, 9),
  (2017, 8, 15, 14),
  (2017, 8, 15, 16)},
 205: {(2017, 8, 18, 16),
  (2017, 8, 18, 19),
  (2017, 8, 19, 10),
  (2017, 8, 19, 15),
  (2017, 8, 20, 11),
  (2017, 8, 20, 16)},
 207: {(2017, 9, 18, 13),
  (2017, 9, 18, 18),
  (2017, 9, 19, 14),
  (2017, 9, 19, 17),
  (2017, 9, 20, 15),
  (2017, 9, 22, 9)},
 208: {(2017, 9, 18, 16),
  (2017, 9, 19, 9),
  (2017, 9, 19, 12),
  (2017, 9, 19, 14),
  (2017, 9, 20, 9),
  (2017, 9, 20, 14),
  (2017, 9, 20, 17),
  (2017, 9, 24, 19),
  (2017, 9, 26, 18),
  (2017, 9, 27, 16),
  (2017, 9, 28, 9),
  (2017, 9, 28, 14)},
 209: {(2017, 9, 29, 14),
  (2017, 9, 29, 16),
  (2017, 9, 29, 19),
  (2017, 9, 30, 12)},
 211: {(2017, 10, 2, 16),
  (2017, 10, 2, 19),
  (2017, 10, 3, 11),
  (2017, 10, 3, 19),
  (2017, 10, 4, 10),
  (2017, 10, 4, 19)

In [301]:
# Construct a list of id + dates for eod
# Use to look 
eod_dates = []
for irow in range(0,eod_complete.shape[0]):
    row = eod_complete.iloc[irow]
    if row['status'] == "MISSED":
        continue
    try:
        time = datetime.datetime.strptime(row['date'], '%m/%d/%Y %H:%M')
    except:
        time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
    if time.hour  == 0 or time.hour == 1:
        date = np.array([row['participant_id'], time.year, time.month, time.day-1])
        date = np.append(date, np.array(row[keys]))
    else:
        date = np.array([row['participant_id'], time.year, time.month, time.day])
        date = np.append(date, np.array(row[keys]))
    eod_dates.append(date)
    
eod_dates = np.asarray(eod_dates)
eod_dates


array([[201, 2017, 6, ..., 0.0, 0.0, 0.0],
       [201, 2017, 6, ..., 0.0, 0.0, 0.0],
       [203, 2017, 8, ..., 0.0, 0.0, 0.0],
       ...,
       [236, 2018, 7, ..., 0.0, 0.0, 0.0],
       [237, 2018, 7, ..., 0.0, 0.0, 0.0],
       [237, 2018, 7, ..., 0.0, 0.0, 0.0]], dtype=object)

In [459]:
# For participants with both EC and EOD measurements,
# on days when you give both, we ask whether they agree,
# up to the current hour, or +- 1 hour in either direction.
# The +-1 is max/min by 8AM and 8PM respectively.
matching_counts = []
max_iloc = 15; min_iloc = 4
for id in set(days_smoked.keys()) & set(eod_dates[:,0]):
    eod_dates_id = np.where(eod_dates[:,0] == id) 
    eod_dates_subset = eod_dates[eod_dates_id[0],:]
    total_count_id = 0
    hour_count_id_true = 0
    twohour_count_id_true = 0
    for ec_time in days_smoked[id]:
        row_iloc = np.where((eod_dates_subset[:,1:4] == ec_time[0:3]).all(axis=1))[0]
        if not row_iloc.size > 0:
            continue
        total_count_id+=1
        row = eod_dates_subset[row_iloc][0]
        ec_iloc = range(8,20).index(ec_time[3])+4
        if row[ec_iloc]==1:
            hour_count_id_true+=1
        if any(row[range(max(min_iloc, ec_iloc-1), min(max_iloc, ec_iloc+1)+1)] == 1):
            twohour_count_id_true+=1
    matching_counts.append(np.array([total_count_id, hour_count_id_true, twohour_count_id_true], dtype='f'))

matching_counts = np.asarray(matching_counts)

matching_counts = np.delete(matching_counts, (np.where(matching_counts[:,0] == 0)[0][0]), axis=0)

fraction_per_id_onehour = np.divide(matching_counts[:,1],matching_counts[:,0])
fraction_per_id_twohour = np.divide(matching_counts[:,2],matching_counts[:,0])

aggregate_matching_counts = np.sum(matching_counts, axis=0)

aggregate_frac_onehour = aggregate_matching_counts[1]/aggregate_matching_counts[0]
aggregate_frac_twohour = aggregate_matching_counts[2]/aggregate_matching_counts[0]

print 'Current hour only:'
print 'Aggregated data, Fraction agreement between EC and EOD: %s' % (np.round(aggregate_frac_onehour,3))
print 'Mean of Fraction agreement across indidivuals: %s' % (np.round(np.mean(fraction_per_id_onehour),3))
print 'Standard deviation of Fraction agreement across indidivuals: %s' %  (np.round(np.std(fraction_per_id_onehour),3))
print
print 'Plus-minus one hour:'
print 'Aggregated data, Fraction agreement between EC and EOD: %s' % (np.round(aggregate_frac_twohour,3))
print 'Mean of Fraction agreement across indidivuals: %s' % (np.round(np.mean(fraction_per_id_twohour),3))
print 'Standard deviation of Fraction agreement across indidivuals: %s' %  (np.round(np.std(fraction_per_id_twohour),3))



Current hour only:
Aggregated data, Fraction agreement between EC and EOD: 0.544
Mean of Fraction agreement across indidivuals: 0.467
Standard deviation of Fraction agreement across indidivuals: 0.322

Plus-minus one hour:
Aggregated data, Fraction agreement between EC and EOD: 0.841
Mean of Fraction agreement across indidivuals: 0.745
Standard deviation of Fraction agreement across indidivuals: 0.34


In [463]:
# Compute an anova decomposition using the bernoulli likelihood
# This will test if there are significant differences across
# individuals.

llik_onehour = 0; llik_twohour = 0
for i in range(0, fraction_per_id_onehour.size):
    num_ones_onehour = matching_counts[i,1]
    num_zeros_onehour = matching_counts[i,0] - matching_counts[i,1]
    if num_ones_onehour > 0.0:
        llik_onehour += np.multiply(num_ones_onehour, np.log(fraction_per_id_onehour[i]))
    if num_zeros_onehour > 0.0:
        llik_onehour += np.multiply(num_zeros_onehour, np.log(1-fraction_per_id_onehour[i]))
    num_ones_twohour = matching_counts[i,2]
    num_zeros_twohour = matching_counts[i,0] - matching_counts[i,2]
    if num_ones_twohour > 0.0:
        llik_twohour += np.multiply(num_ones_twohour, np.log(fraction_per_id_twohour[i]))
    if num_zeros_twohour > 0.0:
        llik_twohour += np.multiply(num_zeros_twohour, np.log(1-fraction_per_id_twohour[i]))

agg_num_ones = aggregate_matching_counts[1]
agg_num_zeros = aggregate_matching_counts[0] - aggregate_matching_counts[1]
agg_llik_onehour = agg_num_ones*np.log(aggregate_frac_onehour)+agg_num_zeros*np.log(1-aggregate_frac_onehour)

D_onehour = -2*agg_llik_onehour + 2*llik_onehour

agg_num_ones_twohour = aggregate_matching_counts[2]
agg_num_zeros_twohour = aggregate_matching_counts[0] - aggregate_matching_counts[2]
agg_llik_twohour = agg_num_ones_twohour*np.log(aggregate_frac_twohour)+agg_num_zeros_twohour*np.log(1-aggregate_frac_twohour)

D_twohour = -2*agg_llik_twohour + 2*llik_twohour

from scipy.stats import chi2
n = aggregate_matching_counts[0]
k = matching_counts.shape[0]
df = k-1

print 'ANOVA p-value for current hour: %s' % (1-chi2.cdf(D_onehour, df))
print 'ANOVA p-value for plus-minus one hour: %s' % (1-chi2.cdf(D_twohour, df))



ANOVA p-value for current hour: 2.422465948948016e-06
ANOVA p-value for plus-minus one hour: 8.337050984019712e-07
